In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-data/objects.bin
/kaggle/input/house-data/m_1ce96d9d245ca490.0000
/kaggle/input/house-data/m_1ce96d9d245ca490.sidx
/kaggle/input/house-data/dir_archive.ini
/kaggle/input/house-data/m_1ce96d9d245ca490.frame_idx


In [4]:
!pip install turicreate
import turicreate as tc

     |████████████████████████████████| 91.9 MB 2.7 kB/s  eta 0:00:01
     |████████████████████████████████| 322 kB 16.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 20.0 MB/s eta 0:00:01
     |████████████████████████████████| 86.3 MB 85 kB/s s eta 0:00:01��████████▌    | 74.1 MB 17.7 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 3.2 MB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 449 kB 23.7 MB/s eta 0:00:01
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320848 sha256=c6ce0b37c50f5b0b7ff2a328d01b8873b2abf9a1d47c67e5cbcc219b81181a1f
  Stored in directory: /root/.cache/pip/wheels/71/74/53/d5ceb7c5ee7a168c7d106041863e71ac3273f4a4677743a284
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7539 sha256=5541c687b132785e74d58eed55ad5a7b31dd740f277b39c11bde21cf2d6dec00
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803

  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Attempting uninstall: gast
    Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow-probability 0.11.0 requires gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
librosa 0.8.0 requires resampy>=0.2.2, but you'll have resampy 0.2.1 which is incom

In [5]:
sales= tc.SFrame('../input/house-data')

# LASSO (coordinate descent)
We will implement your very own LASSO solver via coordinate descent. 

##Write a function to normalize features
##Implement coordinate descent for LASSO
##Explore effects of L1 penalty


In [6]:
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

In [7]:
import numpy as np # note this allows us to refer to numpy as np instead 


In [8]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [9]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)

    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: sqft_living is very large overall compared to bedrooms, for instance. As a result, weight for sqft_living would be much smaller than weight for bedrooms. This is problematic because "small" weights are dropped first as l1_penalty goes up.

To give equal considerations for all features, we need to normalize features as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [10]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print (X)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


In [11]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms)

[ 5. 13. 17.]


In [12]:
print (X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


In [13]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix/norms
    return (normalized_features, norms)

In [14]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features
We seek to obtain a sparse set of weights by minimizing the LASSO cost function

SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
(By convention, we do not include w[0] in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. That is, we look for

argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
where all weights other than w[i] are held to be constant. We will optimize one w[i] at a time, circling through the weights multiple times.

#Pick a coordinate i
#Compute w[i] that minimizes the cost function SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)
#Repeat Steps 1 and 2 for all coordinates, multiple times
For this notebook, we use cyclical coordinate descent with normalized features, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:

           ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
##w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
           └ (ro[i] - lambda/2)     if ro[i] > lambda/2
##where

##ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
Note that we do not regularize the weight of the constant feature (intercept) w[0], so, for this weight, the update is simply:

w[0] = ro[i]

In [15]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [16]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)


In [17]:
weights = np.array([1., 4., 1.])


In [18]:
prediction = predict_output(simple_feature_matrix, weights)


In [19]:
ro = [0 for i in range((simple_feature_matrix.shape)[1])]
for j in range((simple_feature_matrix.shape)[1]):   
    ro[j] = (simple_feature_matrix[:,j] * (output - prediction + (weights[j] * simple_feature_matrix[:,j]))).sum()
print (ro)

[79400300.03492916, 87939470.77299108, 80966698.67596565]


In [20]:
# Return True if value is within the threshold ranges otherwise False
# Looking for range -l1_penalty/2 <= ro <= l1_penalty/2
def in_l1range(value, penalty):
    return ( (value >= -penalty/2.) and (value <= penalty/2.) )

In [22]:
for l1_penalty in [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]:
    print (in_l1range(ro[1], l1_penalty), in_l1range(ro[2], l1_penalty))

False False
False True
False True
True True
True True


# Single Coordinate Descent Step
Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [23]:
 def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] * (output - prediction + (weights[i] * feature_matrix[:,i]))).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = (ro_i + l1_penalty/2.)
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i - l1_penalty/2.)
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [25]:
# should print 0.425558846691
import math
print (lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


# Cyclical coordinate descent
Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:

As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.
Return weights

##IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.

for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

    # use old_weights_i to compute change in coordinate
    ...

In [26]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    D = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    change = np.array(initial_weights) * 0.0
    converged = False

    while not converged:

    # Evaluate over all features
        for idx in range(D):
#             print 'Feature: ' + str(idx)
            # new weight for feature
            new_weight = lasso_coordinate_descent_step(idx, feature_matrix,
                                                       output, weights,
                                                       l1_penalty)
            # compute change in weight for feature
            change[idx] = np.abs(new_weight - weights[idx])
#             print '  -> old weight: ' + str(weights[idx]) + ', new weight: ' + str(new_weight)
#             print '  -> abs change (new - old): ' + str(change[idx])
#             print '  >> old weights: ', weights

            # assign new weight
            weights[idx] = new_weight
#             print '  >> new weights: ', weights
        # maximum change in weight, after all changes have been computed
        max_change = max(change)
#         print '  ** max change: ' + str(max_change)
#         print '--------------------------------------------------'
        if max_change < tolerance:
            converged = True
    return weights

In [27]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [28]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [29]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [30]:
print (weights)

[21624998.36636292 63157246.78545421        0.        ]


In [32]:
prediction =  predict_output(normalized_simple_feature_matrix, weights)
RSS = np.dot(output-prediction, output-prediction)
print ('RSS for normalized dataset = ', RSS)

RSS for normalized dataset =  1630492481484487.5


# Evaluating LASSO fit with more features
Let us split the sales dataset into training and test sets.

In [33]:
train_data,test_data = sales.random_split(.8,seed=0)

In [34]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [35]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

First, learn the weights with l1_penalty=1e7, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights weights1e7, you will need them later.

In [36]:
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e7
tolerance = 1.0

In [37]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

In [38]:
print (weights1e7)

[24429600.60933314        0.                0.         48389174.35227978
        0.                0.          3317511.16271982  7329961.9848964
        0.                0.                0.                0.
        0.                0.        ]


In [40]:
feature_list = ['constant'] + all_features
print (feature_list)

['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


In [42]:
feature_weights1e7 = dict(zip(feature_list, weights1e7))
for k,v in feature_weights1e7.items():
    if v != 0.0:
        print (k, v)

constant 24429600.609333135
sqft_living 48389174.35227978
waterfront 3317511.162719816
view 7329961.984896399


Next, learn the weights with l1_penalty=1e8, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights weights1e8, you will need them later.

In [43]:
l1_penalty=1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

In [44]:
print (weights1e8)

[71114625.75280938        0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.        ]


In [45]:
feature_weights1e8 = dict(zip(feature_list, weights1e8))
for k,v in feature_weights1e8.items():
    if v != 0.0:
        print (k, v)

constant 71114625.75280938


Finally, learn the weights with l1_penalty=1e4, on the training data. Initialize weights to all zeros, and set the tolerance=5e5. Call resulting weights weights1e4, you will need them later. (This case will take quite a bit longer to converge than the others above.)

In [47]:
l1_penalty=1e4
tolerance=5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

In [48]:
print (weights1e4)

[ 77779073.91265215 -22884012.25023361  15348487.08089997
  92166869.69883084  -2139328.0824278   -8818455.54409496
   6494209.73310655   7065162.05053197   4119079.21006765
  18436483.52618778 -14566678.54514349  -5528348.75179429
 -83591746.20730527   2784276.46012858]


In [49]:
feature_weights1e4 = dict(zip(feature_list, weights1e4))
for k,v in feature_weights1e4.items():
    if v != 0.0:
        print (k, v)

constant 77779073.91265215
bedrooms -22884012.250233613
bathrooms 15348487.080899969
sqft_living 92166869.69883084
sqft_lot -2139328.0824277983
floors -8818455.544094957
waterfront 6494209.733106553
view 7065162.050531973
condition 4119079.210067645
grade 18436483.526187785
sqft_above -14566678.545143493
sqft_basement -5528348.75179429
yr_built -83591746.20730527
yr_renovated 2784276.460128576


# Rescaling learned weights
Recall that we normalized our feature matrix, before learning the weights. To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data:

In this case, we must scale the resulting weights so that we can make predictions with original features:

Store the norms of the original features to a vector called norms:
features, norms = normalize_features(features)
Run Lasso on the normalized features and obtain a weights vector
Compute the weights for the original features by performing element-wise division, i.e.
weights_normalized = weights / norms
Now, we can apply weights_normalized to the test data, without normalizing it!
Create a normalized version of each of the weights learned above. (weights1e4, weights1e7, weights1e8).

In [50]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

In [52]:
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms
print (normalized_weights1e7[3])

161.3174562483786


# Evaluating each of the learned models on the test data
Let's now evaluate the three models on the test data:

In [53]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')


In [55]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e7)
RSS = np.dot(test_output-prediction, test_output-prediction)
print ('RSS for model with weights1e7 = ', RSS)

RSS for model with weights1e7 =  275962079909185.3


In [57]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e8)
RSS = np.dot(test_output-prediction, test_output-prediction)
print (""'RSS for model with weights1e8 = ', RSS)

RSS for model with weights1e8 =  537166150034084.9


In [59]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e4)
RSS = np.dot(test_output-prediction, test_output-prediction)
print ('RSS for model with weights1e4 = ', RSS)

RSS for model with weights1e4 =  227781004760225.34
